In [ ]:
import PySimpleGUI as sg
import sqlite3 as sql
import random
import datetime

In [ ]:
global discount_code
global search_city
global search_ptype
pid = None
def login_window():
    layout = [[sg.Text('Welcome to the Property Rental System. Please enter your information.')],
              [sg.Text('ID:',size=(10,1)), sg.Input(size=(10,1), key='id')],
              [sg.Text('Password:',size=(10,1)),  sg.Input(size=(10,1),key = "password")],
              [sg.Button('Login'),sg.Button("Sign Up")]]

    return sg.Window('Property Rental System Login', layout)

def sign_up_window():
    layout = [[sg.Text('Please fill up the information.')],
              [sg.Text('Full Name:',size=(10,1)), sg.Input(size=(30,1), key='name')],
              [sg.Text('ID:',size=(10,1)), sg.Input(size=(30,1), key='id')],
              [sg.Text('Password:',size=(10,1)), sg.Input(size=(30,1), key='password')],
              [sg.Text('User Type:',size=(10,1)), sg.Combo(["owner","customer"],key = "user_type")],
              [sg.Text('Telephone:',size=(10,1)), sg.Input(size=(30,1),key = "pnumber")],
              [sg.Text('Email:',size=(10,1)), sg.Input(size=(30,1),key = "email")],
              [sg.Button('Register'), sg.Button('Sign In')]]
    return sg.Window('Register', layout)

def window_owner():
    property_ids = []
    con = sql.connect('CS281.db')
    cur = con.cursor()
    for row in cur.execute('SELECT PID FROM Property where oid = ?',(login_user_id,)):
        property_ids.append(row)
    con.close()
    layout = [[sg.Text('Welcome ' + login_user_name)],
              [sg.Text('My Properties:'),sg.Combo(property_ids,size=(40,7),key='property_id'),sg.Button("View Property"),sg.Button("Delete Property")],
              [sg.Button("Add Property")],
              [sg.Button('Logout')]]
    window = sg.Window('Owner Window', layout,finalize=True)
    return window

def window_customer(is_search,filtered_properties):
    if not is_search:
        property_ids = []
        con = sql.connect('CS281.db')
        cur = con.cursor()
        for row in cur.execute('SELECT PID FROM Property'):
            property_ids.append(row)
        con.close()
        city_row = [sg.Text('City:',size=(10,1)), sg.Input(size=(10,1), key='city')]
        p_type_row = [sg.Text('Property Type:',size=(10,1)),  sg.Input(size=(10,1),key = "ptype")]
    else:
        property_ids = filtered_properties
        city_row = [sg.Text('City: '+str(search_city))]
        p_type_row = [sg.Text('Property Type: '+str(search_ptype))]

    layout = [[sg.Text('Welcome ' + login_user_name)],
              [sg.Text("Properties:"),sg.Combo(property_ids,size=(40,7),key='property_id'),sg.Button("View Property"),sg.Button("Evaluate Property") ],
              city_row,
              p_type_row,
              [sg.Button("Search Properties")],[sg.Button("Reset Search Filters")],
              [sg.Button('Logout')]]
    window = sg.Window('Customer Window', layout)
    return window
def window_admin():
    property_ids = []
    owner_ids = []
    customer_ids = []
    con = sql.connect('CS281.db')
    cur = con.cursor()
    for row in cur.execute('SELECT PID FROM Property'):
        property_ids.append(row)
    for row in cur.execute('SELECT oid FROM Owner'):
        owner_ids.append(row)
    for row in cur.execute('SELECT cid FROM Customer'):
        customer_ids.append(row)        

    con.close()
    layout = [[sg.Text('Welcome ' + login_user_name)],
              [sg.Text('Properties:'),sg.Combo(property_ids,size=(40,7),key='property_id'),sg.Button("View Property"),sg.Button("Delete Property")],
              [sg.Text('Customers:'),sg.Combo(customer_ids,size=(40,7),key='customer_id'),sg.Button("View Customer"),sg.Button("Delete Customer")],
              [sg.Text('Owners:'),sg.Combo(owner_ids,size=(40,7),key='owner_id'),sg.Button("View Owner"),sg.Button("Delete Owner")],
              [sg.Button("See Availability")],
              [sg.Button('Logout')]]
    window = sg.Window('Admin Window', layout,finalize=True)
    return window
def rate_property(pid):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    global bid
    global rate_pid
    rate_pid = pid
    booked_before= []
    for row in cur.execute("SELECT EXISTS(SELECT 1 FROM Booking WHERE pid=? and cid=?)", (pid, login_user_id)):
        booked_before.append(row)
    if booked_before[0][0]:
        for row in cur.execute("SELECT bid FROM Booking WHERE pid=? and cid=?", (pid, login_user_id)):
            bid = row[0]
            print(bid)
    else:
        print("sldffs")
        sg.Popup("You can't evaluate a property without booking")
        return window_customer(False,cor_properties)
    con.close()

    layout = [
              [sg.Text('Rating:',size=(10,1)), sg.Input(size=(10,1), key='rating')],
              [sg.Text('Comment:',size=(10,1)),  sg.Input(size=(10,1),key = "comment")],
              [sg.Button("Submit Evaluation")]]
    window = sg.Window('Customer Window', layout)
    return window

def update_evaluation(values):
    rating = values["rating"]
    comment = values["comment"]
    con = sql.connect('CS281.db')
    cur = con.cursor()
    eids = []
    for row in cur.execute("SELECT eid from Evaluation"):
        eids.append(row[0])
    eid = create_aid(eids)
    print("bid:" + str(bid))
    print("rating:" + str(rating))
    cur.execute('INSERT OR REPLACE INTO Perform Values(?,?,?)', (login_user_id, bid, eid))
    cur.execute('INSERT OR REPLACE INTO Evaluation Values(?,?,?)', (eid,rating,comment))
    con.commit()
    con.close()

    return True

# def search_window():
#     property_ids = []
#     global comments
#     comments= []
#     con = sql.connect('CS281.db')
#     cur = con.cursor()
#     cor_properties= []
#     for row in cur.execute('SELECT P.PID, E.Rating FROM Property P, Perform Pe, Evaluation E, Booking B where E.eid=Pe.eid and Pe.bid = B.bid and B.pid = P.pid and P.city=? and P.ptype = ? group by P.pid', (search_city, search_ptype, )):
#         cor_properties.append(row)
#         print(row)
#     con.close()
#     layout = [
#               [sg.Text("Properties:"),sg.Combo(cor_properties,size=(40,7),key='property_id'),sg.Button("See Comments")]
#               ]
#     window = sg.Window('Search Window', layout)
#     return window

def search_window():
    property_ids = []
    global comments
    comments= []
    con = sql.connect('CS281.db')
    cur = con.cursor()
    global cor_properties
    cor_properties= []
    for row in cur.execute('SELECT P.PID FROM Property P, Perform Pe, Evaluation E, Booking B where E.eid=Pe.eid and Pe.bid = B.bid and B.pid = P.pid and P.city=? and P.ptype = ? group by P.pid', (search_city, search_ptype, )):
        cor_properties.append(row)
        print(row)
    con.close()
    # layout = [
    #           [sg.Text("Properties:"),sg.Combo(cor_properties,size=(40,7),key='property_id'),sg.Button("See Comments")]
    #           ]
    # window = sg.Window('Search Window', layout)
    return False


def display_comments(pid):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    global cur_pid
    cur_pid = pid
    #rows = []
    ratings=[]
    comments = []
    for row in cur.execute('SELECT E.Comment, E.Rating FROM Property P, Perform Pe, Evaluation E, Booking B where E.eid=Pe.eid and Pe.bid = B.bid and B.pid = P.pid and P.pid=?',(pid,)):
        #pro = tupleToDict(row)
        comment, rating = row
        ratings.append(rating)
        comments.append(comment)
        #rows.append(row)
    con.close()
    #last_row = [sg.Button('See Comments'),sg.Button("See Availability"),sg.Button("Book")]

    layout = [[sg.Text('COMMENTS: ' + str(comments))],
                [sg.Text('RATINGS: ' + str(ratings))]]

    #layout.append(last_row)
    window = sg.Window('Property Window', layout)
    return window

def window_book():
    layout =  [
              [sg.Text('Availability start date: ',size=(20,1)),sg.Input(size=(30,1),key='start_date')],
              [sg.Text('Availability end date: ',size=(20,1)),sg.Input(size=(30,1),key='end_date')],
              [sg.Text('Status : ',size=(20,1)),sg.Input(size=(30,1),key='status')],
              [sg.Button("Finalize booking")]]
    return sg.Window('Booking',layout)

def update_book(values):
    global login_user_id
    #global pid
    con = sql.connect('CS281.db')
    cur = con.cursor()
    start_date = datetime.datetime(int(values['start_date'].strip().split("-")[2]), int(values['start_date'].strip().split("-")[0]), int(values['start_date'].strip().split("-")[1]))
    end_date = datetime.datetime(int(values['end_date'].strip().split("-")[2]), int(values['end_date'].strip().split("-")[0]), int(values['end_date'].strip().split("-")[1]))

    status = values["status"]
    availability_list = {}
    booking_list = {}
    print(cur_pid)
    for row in cur.execute(f'SELECT nightly_price FROM Property WHERE pid = {cur_pid}'):
        nightly_price = row[0]
        nightly_price = int(nightly_price)
    for row in cur.execute(f'SELECT * FROM Availability WHERE pid = {cur_pid}'):
        pro = tupleToDict_av(row)
        start = datetime.datetime(int(pro['start_date'].strip().split("-")[2]), int(pro['start_date'].strip().split("-")[0]), int(pro['start_date'].strip().split("-")[1]))
        end = datetime.datetime(int(pro['end_date'].strip().split("-")[2]), int(pro['end_date'].strip().split("-")[0]), int(pro['end_date'].strip().split("-")[1]))
        availability_list[pro["aid"]] = [start,end,pro['status']]

    for row in cur.execute(f'SELECT * FROM Booking WHERE pid = {cur_pid}'):
        pro = tupleToDict_book(row)
        print(pro["start_date"])
        print(type(pro["start_date"]))
        start = datetime.datetime(int(pro['start_date'].strip().split("-")[2]), int(pro['start_date'].strip().split("-")[0]), int(pro['start_date'].strip().split("-")[1]))
        end = datetime.datetime(int(pro['end_date'].strip().split("-")[2]), int(pro['end_date'].strip().split("-")[0]), int(pro['end_date'].strip().split("-")[1]))
        booking_list[pro["bid"]] = [start,end]
    duration = end_date-start_date

    for bid in booking_list.keys():
        if(booking_list[bid][0]<= start_date <= booking_list[bid][1] or booking_list[bid][0]<= end_date<=booking_list[bid][1]):
            return False

    updated = {}
    new = {}
    aids = list(availability_list.keys())
    print(aids)
    case_no = 0
    for aid in availability_list.keys():
        if(availability_list[aid][0]<start_date<availability_list[aid][1] and end_date>availability_list[aid][1]): #case 1
            print("Case 1")
            case_no = 1
            updated[aid] = [availability_list[aid][0],start_date,availability_list[aid][2]]
            a = create_aid(aids)
            aids.append(a)
            new[a] = [start_date,availability_list[aid][1],status]
            a = create_aid(aids)
            aids.append(a)
            new[a] = [availability_list[aid][1],end_date,status]
        elif(availability_list[aid][0]<end_date<availability_list[aid][1] and start_date<availability_list[aid][0]): #case 2
            print("Case 2")
            case_no = 2
            a = create_aid(aids)
            aids.append(a)
            new[a] = [start_date,availability_list[aid][0],status]
            updated[aid] = [availability_list[aid][0],end_date,status]
            a =  create_aid(aids)
            aids.append(a)
            new[a] = [end_date,availability_list[aid][1],status]
        elif(availability_list[aid][0]<start_date<availability_list[aid][1] and availability_list[aid][0]<end_date<availability_list[aid][1] ): #case 3
            print("Case 3")
            case_no = 3
            updated[aid] = [availability_list[aid][0],start_date,availability_list[aid][2]]
            a = create_aid(aids)
            aids.append(a)
            new[a] = [start_date,end_date,status]
            a =  create_aid(aids)
            aids.append(a)
            new[a] = [end_date,availability_list[aid][1],availability_list[aid][2]]
        else:
            print("Case 4")
            case_no = 4
            a =  create_aid(aids)
            aids.append(a)
            new[a] = [end_date,start_date,status]

    days = round(duration.days)
    #update_availability
    print("Updated",updated)
    print("New",new)
    for aid in updated.keys():
        print(aid)
        start = updated[aid][0]
        start = f"{start.month}-{start.day}-{start.year}"
        end = updated[aid][1]
        end = f"{end.month}-{end.day}-{end.year}"
        status = updated[aid][2]
        cur.execute(f"UPDATE Availability SET status = {status},end_date = {end_date},"
                    f"start_date = {start_date}, pid = {cur_pid},oid = {login_user_id} WHERE aid = {aid}")

    for aid in new.keys():
        start = new[aid][0]
        start = f"{start.month}-{start.day}-{start.year}"
        end = new[aid][1]
        end = f"{end.month}-{end.day}-{end.year}"
        status = new[aid][2]
        print(aid)
        cur.execute('INSERT OR REPLACE INTO Availability Values(?,?,?,?,?,?)', (aid,status,end,start,cur_pid,login_user_id))
        print("Availability is added to the database")

    global total_price
    print(nightly_price)
    print(days)
    total_price = nightly_price * days
    global discount_code
    bids = []
    for row in cur.execute("SELECT bid from Booking"):
        bids.append(row[0])

    bid = create_aid(bids)
    ## discount varsa paid degisir herhalde, bunu farklı yapmak gerekebilir
    cur.execute("INSERT INTO Booking Values(?,?,?,?,?,?,?)", (bid, end,start, total_price, cur_pid, aid, login_user_id))

    con.commit()
    ### here get customer's discount code
    for row in cur.execute("SELECT discount_code FROM Customer Where cid=?", (login_user_id, )):
        discount_code= row[0]
        print(discount_code)
    con.close()
    last_row=[]
    #discount_code = 30
    if discount_code:
         last_row = [sg.Button("Use Discount Code")]
    layout = [[sg.Text('Total price: ' + str(total_price))],[sg.Button('Return to home')],
                last_row]
    window = sg.Window('Price Window', layout)
    return window

def final_price():
    discounted_price = total_price*(100-discount_code)/100
    layout = [[sg.Text('Discounted price: ' + str(discounted_price))], [sg.Button('Return to home')]]
    window = sg.Window('Discount Window', layout)
    return window

def window_property(pid,isowner):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    for row in cur.execute('SELECT * FROM Property where PID =?',(pid,)):
        pro = tupleToDict(row)
    avg_ratings=[]
    for row in cur.execute('SELECT E.Rating FROM Property P, Evaluation E, Perform Pe, Booking B where P.PID =? and P.pid=B.pid and B.bid = Pe.bid and Pe.eid=E.eid ',(pid,)):
           avg_ratings.append(row[0])

    try:
        avg_rating = sum(avg_ratings)/len(avg_ratings)
    except:
        avg_rating = "No rating"
        
    
    global cur_pid
    cur_pid = pid
    #rows = []
    ratings=[]
    comments = []
    for row in cur.execute('SELECT E.Comment, E.Rating FROM Property P, Perform Pe, Evaluation E, Booking B where E.eid=Pe.eid and Pe.bid = B.bid and B.pid = P.pid and P.pid=?',(pid,)):
        #pro = tupleToDict(row)
        comment, rating = row
        ratings.append(rating)
        comments.append(comment)
        #rows.append(row)    
    
        
        
    con.close()
    if isowner == "owner":
        last_row = [sg.Button("Update Availability"),sg.Button("See Availability"),sg.Button("Bookings"),sg.Button("See Comments")]
    elif isowner == "admin":
        last_row = [sg.Button("Change Property"),sg.Button("Change Evaluation")]
    else:
        #customer
        last_row = [sg.Button('See Comments'),sg.Button("See Availability"),sg.Button("Book")]

    layout = [[sg.Text('PID: ' + str(pro["PID"]))],
              [sg.Text('Property Name: ' + pro["pname"])],
              [sg.Text('City: ' + pro["city"])],
              [sg.Text('District: ' + pro["district"])],
              [sg.Text('Capacity: ' + pro["capacity"])],
              [sg.Text('Type: ' + pro["ptype"])],
              [sg.Text('Description: ' + pro["description"])],
              [sg.Text('Nightly Price: ' + str(pro["nightly_price"])+"TL")],
              [sg.Text('Average Rating: ' + str(avg_rating)+"/5")],
              [sg.Text('COMMENTS: ' + str(comments))]]

    layout.append(last_row)
    window = sg.Window('Property Window', layout)
    return window

def view_customer(cid):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    for row in cur.execute('SELECT * FROM Customer where cid =?',(cid,)):
        pro = tupleToDict_customer(row)
    con.close()
    layout = [[sg.Text('Customer Name: ' + str(pro["cname"]))],
              [sg.Text('CID: ' + str(pro["cid"]))],
              [sg.Text('Customer email: ' + pro["cemail"])]]
    window = sg.Window('Customer Window', layout)
    return window
def view_owner(oid):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    for row in cur.execute('SELECT * FROM Owner where oid =?',(oid,)):
        pro = tupleToDict_owner(row)
    con.close()
    layout = [[sg.Text('Owner Name: ' + str(pro["oname"]))],
              [sg.Text('OID: ' + str(pro["oid"]))],
              [sg.Text('Owner email: ' + pro["oemail"]),
              [sg.Text('Owner gsmn: ' + str(pro["gsmn"]))]]]
    window = sg.Window('Owner Window', layout)
    return window  
#def change_evaluation_to_table(data,eid):
#    con = sql.connect('CS281.db')
#    cur = con.cursor()
#    cur.execute(f"UPDATE Evaluation SET Rating = {data["rating"]},Comment = {data["comment"]},"
#            f"WHERE eid = {eid}")
#    con.close()
#def change_property():
    
def comment_window(isowner = False):
    if isowner:
        comment_button =  "Reply Comments"
    else:
        comment_button = "Comment"

def button_login(values):

    global login_user_id
    global login_user_name
    global login_type
    global window

    userid = values['id']
    upass = values['password']
    if userid == '':
        sg.popup('ID cannot be empty')
        return False
    elif upass == '':
        sg.popup('Password cannot be empty')
        return False
    else:
        con = sql.connect('CS281.db')
        cur = con.cursor()
        # first check if this is a valid user
        cur.execute('SELECT * FROM User WHERE uid = ? and password = ?', (userid,upass))
        row = cur.fetchone()
        if row is None:
            sg.popup('ID or password is wrong!')
            con.close()
            return False
        else:
            # this is some existing user, let's keep the ID of this user in the global variable
            login_user_id = row[0]

            # we will use the name in the welcome message
            login_user_name = row[3]
            login_type = row[2]
        con.close()

        return True

def button_search(values):
    global search_city
    global search_ptype
    search_city = values['city']
    search_ptype = values['ptype']
    return True


def tupleToDict(row):
    pro = {}
    pro["PID"] = row[0]
    pro["pname"] = row[1]
    pro["city"] = row[2]
    pro["district"] = row[3]
    pro["capacity"] = row[4]
    pro["ptype"] = row[5]
    pro["description"] = row[6]
    pro["nightly_price"] = row[7]
    pro["aid"] = row[8]
    pro["oid"] = row[9]
    return pro
def tupleToDict_av(row):
    pro = {}
    pro["aid"] = row[0]
    pro["status"] = row[1]
    pro["end_date"] = row[2]
    pro["start_date"] = row[3]
    pro["pid"] = row[4]
    pro["oid"] = row[5]
    return pro
def tupleToDict_book(row):
    pro = {}
    pro["bid"] = row[0]
    pro["end_date"] = row[1]
    pro["start_date"] = row[2]
    pro["pid"] = row[3]
    pro["aid"] = row[4]
    return pro
def tupleToDict_belong(row):
    pro = {}
    pro["bid"] = row[0]
    pro["pid"] = row[1]
    pro["aid"] = row[2]
    return pro
def tupleToDict_owner(row):
    pro = {}
    pro["oname"] = row[0]
    pro["oid"] = row[1]
    pro["oemail"] = row[2]
    pro["gsmn"] = row[3]
    return pro
def tupleToDict_customer(row):
    pro = {}
    pro["cname"] = row[0]
    pro["cid"] = row[1]
    pro["cemail"] = row[2]
    return pro


def id_exists(id):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    ids = cur.execute('SELECT uid FROM User')
    if (id in ids):
        return True
    con.close()
    return False

def write_user_to_table(data):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    cur.execute('INSERT INTO User Values(?,?,?,?)', (data["id"],data["password"],data["user_type"],data["name"]))
    if(data["user_type"] == "owner"):
        cur.execute('INSERT INTO Owner Values(?,?,?,?)', (data["name"],data["id"],data["email"],data["pnumber"]))
    else:
        cur.execute('INSERT INTO Customer Values(?,?,?)', (data["name"],data["id"],data["email"]))
    print(f"User {data['name']} is written to database")
    con.commit()
    con.close()

def delete_property():
    global pid
    con = sql.connect('CS281.db')
    cur = con.cursor()
    cur.execute(f'DELETE FROM Property WHERE pid = {pid}')
    con.commit()
    con.close()
    
    
def window_change_evaluation():
    layout = [[sg.Text('Please fill up the property information.')],
              [sg.Text('Rating:',size=(10,1)), sg.Input(size=(30,1), key='rating')],
              [sg.Text('Comment:',size=(10,1)), sg.Input(size=(30,1), key='comment')],
              [sg.Button('Update The Evaluation')]]
    return sg.Window('Change Evaluation', layout)
    
def window_change_property():
    layout = [[sg.Text('Please fill up the property information.')],
              [sg.Text('Property Name:',size=(10,1)), sg.Input(size=(30,1), key='pname')],
              [sg.Text('City:',size=(10,1)), sg.Input(size=(30,1), key='city')],
              [sg.Text('District:',size=(10,1)), sg.Input(size=(30,1), key='district')],
              [sg.Text('Capacity:',size=(10,1)), sg.Combo(["1+1","2+1","3+1","4+1","5+1"],size=(40,7),key='capacity')],
              [sg.Text('Property Type:',size=(10,1)),sg.Combo(["Apartment","Room"],size=(40,7),key='ptype')],
              [sg.Text('Nightly Price:',size=(10,1)),sg.Input(size=(30,1),key='nightly_price')],
              [sg.Text('Description : ',size=(10,1)),sg.Input(size=(30,1),key='description')],
              [sg.Button('Update The Property')]]
    return sg.Window('Change Property', layout)

def window_add_property():
    layout = [[sg.Text('Please fill up the property information.')],
              [sg.Text('Property Name:',size=(10,1)), sg.Input(size=(30,1), key='pname')],
              [sg.Text('Property ID:',size=(10,1)), sg.Input(size=(30,1), key='pid')],
              [sg.Text('City:',size=(10,1)), sg.Input(size=(30,1), key='city')],
              [sg.Text('District:',size=(10,1)), sg.Input(size=(30,1), key='district')],
              [sg.Text('Capacity:',size=(10,1)), sg.Combo(["1+1","2+1","3+1","4+1","5+1"],size=(40,7),key='capacity')],
              [sg.Text('Property Type:',size=(10,1)),sg.Combo(["Apartment","Room"],size=(40,7),key='ptype')],
              [sg.Text('Nightly Price:',size=(10,1)),sg.Input(size=(30,1),key='nightly_price')],
              [sg.Text('Availability start date: ',size=(10,1)),sg.Input(size=(30,1),key='start_date')],
              [sg.Text('Availability end date: ',size=(10,1)),sg.Input(size=(30,1),key='end_date')],
              [sg.Text('Status : ',size=(10,1)),sg.Input(size=(30,1),key='status')],
              [sg.Text('Aid : ',size=(10,1)),sg.Input(size=(30,1),key='aid')],
              [sg.Text('Description : ',size=(10,1)),sg.Input(size=(30,1),key='description')],
              [sg.Button('Register Property')]]
    return sg.Window('Add Property', layout)

def write_property_to_table(data):
    #print(login_user_id)
    con = sql.connect('CS281.db')
    cur = con.cursor()
    cur.execute('INSERT INTO Availability Values(?,?,?,?,?,?)', (data["aid"],data["status"],data["end_date"],data["start_date"],data["pid"],login_user_id))
    cur.execute('INSERT INTO Property Values(?,?,?,?,?,?,?,?,?,?)', (data["pid"],data["pname"],data["city"],data["district"],data["capacity"],data["ptype"],data["description"],data["nightly_price"],data["aid"],login_user_id))
    print(f" {data['pname']} is written to database")
    con.commit()
    con.close()

def window_availability():
    layout =  [
              [sg.Text('Availability start date: ',size=(20,1)),sg.Input(size=(30,1),key='start_date')],
              [sg.Text('Availability end date: ',size=(20,1)),sg.Input(size=(30,1),key='end_date')],
              [sg.Text('Status : ',size=(20,1)),sg.Input(size=(30,1),key='status')],
              [sg.Button("Update")]]
    return sg.Window('Update availability',layout)

def update_availability(values):
    global login_user_id
    global pid
    con = sql.connect('CS281.db')
    cur = con.cursor()
    start_date = datetime.datetime(int(values['start_date'].strip().split("-")[2]), int(values['start_date'].strip().split("-")[0]), int(values['start_date'].strip().split("-")[1]))
    end_date = datetime.datetime(int(values['end_date'].strip().split("-")[2]), int(values['end_date'].strip().split("-")[0]), int(values['end_date'].strip().split("-")[1]))

    status = values["status"]
    availability_list = {}
    booking_list = {}
    print(cur_pid)
    for row in cur.execute(f'SELECT * FROM Availability WHERE pid = {cur_pid}'):
        pro = tupleToDict_av(row)
        start = datetime.datetime(int(pro['start_date'].strip().split("-")[2]), int(pro['start_date'].strip().split("-")[0]), int(pro['start_date'].strip().split("-")[1]))
        end = datetime.datetime(int(pro['end_date'].strip().split("-")[2]), int(pro['end_date'].strip().split("-")[0]), int(pro['end_date'].strip().split("-")[1]))
        availability_list[pro["aid"]] = [start,end,pro['status']]

    for row in cur.execute(f'SELECT * FROM Booking WHERE pid = {cur_pid}'):
        pro = tupleToDict_book(row)
        print(pro["start_date"])
        print(type(pro["start_date"]))
        start = datetime.datetime(int(pro['start_date'].strip().split("-")[2]), int(pro['start_date'].strip().split("-")[0]), int(pro['start_date'].strip().split("-")[1]))
        end = datetime.datetime(int(pro['end_date'].strip().split("-")[2]), int(pro['end_date'].strip().split("-")[0]), int(pro['end_date'].strip().split("-")[1]))
        booking_list[pro["bid"]] = [start,end]

    for bid in booking_list.keys():
        if(booking_list[bid][0]<= start_date <= booking_list[bid][1] or booking_list[bid][0]<= end_date<=booking_list[bid][1]):
            return False

    updated = {}
    new = {}
    aids = list(availability_list.keys())
    case_no = 0

    for aid in availability_list.keys():
        if(availability_list[aid][0]<start_date<availability_list[aid][1] and end_date>availability_list[aid][1]): #case 1
            print("Case 1")
            case_no = 1
            updated[aid] = [availability_list[aid][0],start_date,availability_list[aid][2]]
            a = create_aid(aids)
            aids.append(a)
            new[a] = [start_date,availability_list[aid][1],status]
            a = create_aid(aids)
            aids.append(a)
            new[a] = [availability_list[aid][1],end_date,status]
        elif(availability_list[aid][0]<end_date<availability_list[aid][1] and start_date<availability_list[aid][0]): #case 2
            print("Case 2")
            case_no = 2
            a = create_aid(aids)
            aids.append(a)
            new[a] = [start_date,availability_list[aid][0],status]
            updated[aid] = [availability_list[aid][0],end_date,status]
            a =  create_aid(aids)
            aids.append(a)
            new[a] = [end_date,availability_list[aid][1],status]
        elif(availability_list[aid][0]<start_date<availability_list[aid][1] and availability_list[aid][0]<end_date<availability_list[aid][1] ): #case 3
            print("Case 3")
            case_no = 3
            updated[aid] = [availability_list[aid][0],start_date,availability_list[aid][2]]
            a = create_aid(aids)
            aids.append(a)
            new[a] = [start_date,end_date,status]
            a =  create_aid(aids)
            aids.append(a)
            new[a] = [end_date,availability_list[aid][1],availability_list[aid][2]]
        else:
            print("Case 4")
            case_no = 4
            a =  create_aid(aids)
            aids.append(a)
            new[a] = [end_date,start_date,status]

    #update_availability
    print("Updated",updated)
    print("New",new)
    for aid in updated.keys():
        start = updated[aid][0]
        start = f"{start.month}-{start.day}-{start.year}"
        end = updated[aid][1]
        end = f"{end.month}-{end.day}-{end.year}"
        status = updated[aid][2]
        cur.execute(f"UPDATE Availability SET status = {status},end_date = {end_date},"
                    f"start_date = {start_date}, pid = {cur_pid},oid = {login_user_id} WHERE aid = {aid}")

    for aid in new.keys():
        start = new[aid][0]
        start = f"{start.month}-{start.day}-{start.year}"
        end = new[aid][1]
        end = f"{end.month}-{end.day}-{end.year}"
        status = new[aid][2]
        cur.execute('INSERT INTO Availability Values(?,?,?,?,?,?)', (aid,status,end,start,cur_pid,login_user_id))
        print("Availability is added to the database")
    con.commit()
    con.close()
    return True

def create_aid(aids):
    aid = random.randint(100,999)
    while aid in aids:
        aid = random.randint(100,999)
    return aid
def see_availability(pid):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    availability_list = []
    for row in cur.execute(f'SELECT * FROM Availability WHERE pid = {pid} ORDER BY start_date'):
        pro = tupleToDict_av(row)
        availability_list.append(f"Between these dates: {pro['start_date']} - {pro['end_date']}, property is {pro['status']}")

    con.close()
    layout = [[sg.Text('PID: ' + str(pro["pid"])+ "\t\t")],
              [sg.Listbox(availability_list,select_mode='extended', key='fac', size=(100, 8))]]
    window = sg.Window('Availability Window', layout)
    return window

def see_bookings(pid):
    con = sql.connect('CS281.db')
    cur = con.cursor()
    for row in cur.execute(f'SELECT * FROM Booking WHERE pid = {pid}'):
        prob = tupleToDict_book(row)
    for row_2 in cur.execute(f'SELECT * FROM Availability WHERE pid = {pid}'):
        pro2 = tupleToDict_av(row_2)
    for row_3 in cur.execute(f'SELECT * FROM Belongs_to WHERE pid = {pid}'):
        pro3 = tupleToDict_belong(row_3)
    cname_list = []
    for row_4 in cur.execute(f'SELECT c.cname FROM Belongs_to bt,Booking b,Perform p,Customer c WHERE bt.pid = {pid} and bt.bid = b.bid and b.bid = p.bid and p.cid = c.cid'):
        cname_list.append(row_4)
    con.close()
    if (pro2["status"] == "AVAILABLE" or pro2["status"] == "available"):
        isAvailable = True
    else:
        isAvailable = False
    #prob = probo
    print(cname_list)
    if isAvailable == False:
        layout = [[sg.Text('PID: ' + str(prob["pid"])+ "\t\t")],
                  [sg.Text('The property is booked...')],
                  [sg.Text('Booking Dates: ')],
                  [sg.Text('Start date: ' + prob["start_date"]+ "\t\t" )],
                  [sg.Text('End date: ' + prob["end_date"]+ "\t\t" )],
                  [sg.Text('Booked by: '+str(cname_list[0]))]]
    else:
        layout = [[sg.Text('No booking on selected property \t\t')]]

    """
    if isAvailable == True:
        layout = [[sg.Text('PID: ' + str(pro2["pid"])+ "\t\t")],
                  [sg.Text('Property is Available in: ')],
                  [sg.Text('Start date: ' + prob["start_date"]+ "\t\t" )],
                  [sg.Text('End date: ' + prob["end_date"]+ "\t\t" )]]
    """
    window = sg.Window('Bookings Window', layout)
    return window

In [ ]:
con = sql.connect('CS281.db')
global cor_properties
cor_properties = []
cur = con.cursor()
# open the first window
windows = {"main" : None, "side" : []}
window = login_window()
windows["main"] = window
isfirst=True
while True:
    event, values = window.read()
    if event == 'Login':
        if(button_login(values)):
            window.close()
            if(login_type =="owner"):
                window = window_owner()
                print("Window Owner")
            elif(login_type == "admin"):
                window = window_admin()
                print("window Admin")
            else:
                window = window_customer(False,cor_properties)
                print("Window Customer")
            windows["main"] = window
            windows["side"] = []
    elif event == "Return to home":
            window.close()
            if(login_type =="owner"):
                window = window_owner()
                print("Window Owner")
            else:
                window = window_customer(False,cor_properties)
            windows["main"] = window
            windows["side"] = []
    elif event == "View Property":
        viewed_pid = list(values["property_id"])[0]
        pid = viewed_pid
        cur_pid = viewed_pid
        print(f"Login type {login_type}")
        window = window_property(viewed_pid,login_type)
        windows["side"].append(window)
    elif event == "View Owner":
        viewed_oid = list(values["owner_id"])[0]
        window = view_owner(viewed_oid)
        windows["side"].append(window)
    elif event == "View Customer":
        viewed_cid = list(values["customer_id"])[0]
        window = view_customer(viewed_cid)
        windows["side"].append(window)
    elif event == "Change Property":
        window = window_change_property()
        windows["side"].append(window) 
    elif event == "Change Evaluation":
        window = window_change_evaluation()
        windows["side"].append(window)
    elif event == "Update The Evaluation":
        eid = 1
        change_evaluation_to_table(values,eid)
        window.close()
    elif event == "Reset Search Filters":
        isfirst = True
        window.close()
        window = window_customer(False,cor_properties)
        windows["main"] = window
    elif event == "See Availability":
        window = see_availability(viewed_pid)
        windows["side"].append(window)
    elif event == "Logout":
        window.close()
        window = login_window()
    elif event == "Search Properties":
        button_search(values)
        search_window()
        window.close()
        window = window_customer(True,cor_properties)
        windows["main"] = window
    elif event=="Use Discount Code":
        window.close()
        window = final_price()
        windows["side"].append(window)
    elif event == "Book":
        window = window_book()
        windows["side"].append(window)
    elif event == "Evaluate Property":
        viewed_pid = list(values["property_id"])[0]
        rate_pid = viewed_pid
        window = rate_property(rate_pid)
        windows["side"].append(window)
    elif event == "Submit Evaluation":
        success = update_evaluation(values)
        if success:
            window.close()
    elif event == "Finalize booking":
        #window.close()
        window = update_book(values)
        windows["side"].append(window)
        #windows["side"].append(window)
    elif event == "See Comments":
        #viewed_pid = list(values["property_id"])[0]
        window = display_comments(cur_pid)
        windows["side"].append(window)
    elif event == "Bookings":
        window = see_bookings(viewed_pid)
        windows["side"].append(window)

    elif event == "Sign Up":
        window.alpha_channel = 0
        window = sign_up_window()
        windows["side"].append(window)
    elif event == "Register":
        id = values["id"]
        if(not id_exists(id)):
            write_user_to_table(values)
        else:
            sg.popup("This ID exists!!")
    elif event == "Sign In":
        window = login_window()
        windows["main"] = window
    elif event == "Delete Property":
        pid = list(values["property_id"])[0]
        print(f"property to be deleted {pid}")
        delete_property()
        window.close()
        if login_type == "admin":
            window = window_admin()
        else:
            window = window_owner()
    elif event == "Register Property":
        write_property_to_table(values)
        window.close()
        window = window_owner()
        windows["main"] = window
        windows["side"] = []
    elif event == "Add Property":
        window = window_add_property()
        windows["side"].append(window)
    elif event == "Update Availability":
        print("HEREEEEEEEEEEEEEEEEEEEEEE")
        window = window_availability()
        windows["side"].append(window)
    elif event == "Update":
        if not update_availability(values):
            sg.popup("Update cannot be done!")
        else:
            window.close()
            window = windows["side"][0]
    elif event == sg.WIN_CLOSED:
        if len(windows["side"]) > 1 :
            window = windows["side"][0]
            windows["side"] = [window]
        elif len(windows["side"]) == 1:
            window = windows["main"]
            windows["side"] = []
        elif len(windows["side"]) == 0:
            break


window.close()

con.commit()
con.close()